# Tutorial for HPS multiple DDX4 

In [6]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit
import mdtraj
try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../../')
from openabc.forcefields.parsers import HPSParser
from openabc.forcefields import HPSModel
from openabc.utils.helper_functions import build_straight_CA_chain, write_pdb
from openabc.utils.insert import insert_molecules

# set simulation platform
platform_name = 'CPU'

Parse a single DDX4.

In [7]:
sequence = 'MGDEDWEAEINPHMSSYVPIFEKDRYSGENGDNFNRTPASSSEMDDGPSRRDHFMKSGFASGRNFGNRDAGECNKRDNTSTMGGFGVGKSFGNRGFSNSRFEDGDSSGFWRESSNDCEDNPTRNRGFSKRGGYRDGNNSEASGPYRRGGRGSFRGCRGGFGLGSPNNDLDPDECMQRTGGLFGSRRPVLSGTGNGDTSQSRSGSGSERGGYKGLNEEVITGSGKNSWKSEAEGGES'
ca_pdb = 'init_DDX4_CA.pdb'
ca_atoms = build_straight_CA_chain(sequence, r0=0.38)
write_pdb(ca_atoms, ca_pdb)
protein_parser = HPSParser(ca_pdb)

Parse molecule with default settings.


Prepare the initial configuration.

In [8]:
# insert molecules into the simulation box randomly
n_mol = 10
if not os.path.exists('start.pdb'):
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[50, 50, 50])


Use the Urry scale optimal parameter ($\mu=1$ and $\Delta=0.08$) and run the simulation. 

In [9]:
protein = HPSModel()
for i in range(n_mol):
    protein.append_mol(protein_parser)
top = app.PDBFile('start.pdb').getTopology()
init_coord = app.PDBFile('start.pdb').getPositions()
protein.create_system(top)
protein.add_protein_bonds(force_group=1)
protein.add_contacts('Urry', mu=1, delta=0.08, force_group=2)
protein.add_dh_elec(force_group=3)
temperature = 300*unit.kelvin
friction_coeff = 0.01/unit.picosecond # use smaller friction coefficient to accelerate dynamics
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
platform_name = 'CPU'
protein.set_simulation(integrator, platform_name, init_coord=init_coord)
protein.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output_multi_DDX4.dcd'
protein.add_reporters(output_interval, output_dcd)
protein.simulation.context.setVelocitiesToTemperature(temperature)
protein.simulation.step(500)

Add protein bonds.
Add nonbonded contacts.
Use Urry hydropathy scale.
Scale factor mu = 1 and shift delta = 0.08.
Add Debye-Huckel electrostatic interactions.
Set Debye length as 1 nm.
Set water dielectric as 80.0.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,2600.623669551789,5956.331977768012,8556.9556473198,202.45360304718304,0
200,2.0000000000000013,3081.0428007288133,5521.044159836187,8602.086960565,187.6583251090539,323
300,2.99999999999998,2912.410144432957,5789.632299945443,8702.0424443784,196.7875403549168,324
400,3.9999999999999587,2817.8429803771583,5934.708888941865,8752.551869319022,201.7186419573374,323
500,4.999999999999938,2649.3505835466144,6163.174527363165,8812.525110909779,209.48410765728644,321


In [10]:
# view trajectory
try:
    traj = mdtraj.load_dcd(output_dcd, top='start.pdb')
    nglview.show_mdtraj(traj)
    print('Show DDX4 simulation trajectory.')
except NameError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

Show DDX4 simulation trajectory.
